Throughout the assignment we will define $x:=x_1$, $y:=x_2$, and $\bold x:=[x,y]^T$ to make the assignment concistent as subscripts can not be used when defining variables in SymPy

The Rosenbrock function is defined as 
$$
f(\bold x) = 100(y-x^2)^2+(1-x)^2
$$

# 1

To compute the gradient, we compute the partial derivatives for *f* w.r.t. *x* and *y*.
$$
\frac{d}{dx}f(\bold x)=2\cdot 100(y-x^2)\cdot (-2x)+2(1-x)\cdot (-1)=-400x(y-x^2)-2(1-x)
$$
$$
\frac{d}{dx}f(\bold y)= 2\cdot 100(y-x^2)\cdot=200(y-x^2)
$$

We now have all we need to construct the gradient
$$
\nabla f(\bold x)=\begin{bmatrix}\frac{d}{dx}f(\bold x) \\ \frac{d}{dy}f(\bold x)\end{bmatrix} = \begin{bmatrix} -400x(y-x^2)-2(1-x)\\200(y-x^2)\end{bmatrix}
$$

The Hessian contains the second order derivatives
$$
H = \begin{bmatrix}\frac{d}{dx^2}f(\bold x) & \frac{d}{dxdy}f(\bold x)\\
\frac{d}{dydx}f(\bold x) & \frac{d}{dy^2}f(\bold x)\end{bmatrix}

$$\frac{d}{dx^2}f(\bold x)=-400(y-x^2)-400x\cdot (-2x)+2=-400(y-x^2)+800x^2+2=1200x^2-400y+2$$
$$\frac{d}{dy^2}f(\bold x)=200$$

Luckily, the Hessian is symmetric so we only need to calculate one of the mixed derivatives.

$$\frac{d}{dxdy}f(\bold x)=-400x$$

We can now state the Hessian
$$
H = \begin{bmatrix}1200x^2-400y+2 & -400x\\
-400x & 200
\end{bmatrix}
$$

# 2

To ensure that $x^*=[1,1]^T$ 